## Pandas Primer part 1
-------------

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns; sns.set()
import math
import zmq
import statsmodels.api as sm
import matplotlib.pyplot as plt
import quandl as ql
ql.ApiConfig.api_key= "9Djob1GoNyL-pEK4-vEk"
import pyodbc
cnxn = pyodbc.connect(driver='{SQL Server Native Client 11.0}',
                      server='LAPTOP-2DK08FTD\SQLEXPRESS', 
                      database='AdventureWorksDW2017',               
                      trusted_connection='yes')

## Basic Pandas

In a sense, **`pandas` is built "on top" of `NumPy`**. For example, `NumPy` universal functions will generally work on `pandas` objects as well. We therefore import both to begin with.

On a rather fundamental level, the `DataFrame` class is designed to manage **indexed and labeled data**.

### Introduction to Data Analysis with Pandas

In [2]:
sql = """SELECT FF.FinanceKey,DD.FullDateAlternateKey,DO.OrganizationName,DDep.DepartmentGroupName,DS.ScenarioName, DA.AccountDescription, DA.AccountType,FF.Amount

FROM FactFinance as FF JOIN DimDate as DD ON FF.DateKey = DD.DateKey
JOIN DimOrganization AS DO
ON FF.OrganizationKey = DO.OrganizationKey
JOIN DimDepartmentGroup AS DDep
ON FF.DepartmentGroupKey = DDep.DepartmentGroupKey
JOIN DimScenario AS DS
ON FF.ScenarioKey = DS.ScenarioKey
JOIN DimAccount AS DA
ON FF.AccountKey = DA.AccountKey
"""

In [3]:
data = pd.read_sql(sql,cnxn)
data.head()

,FinanceKey,FullDateAlternateKey,OrganizationName,DepartmentGroupName,ScenarioName,AccountDescription,AccountType,Amount
0,1,2010-12-29,Northeast Division,Corporate,Actual,Salaries,Expenditures,22080.0
1,2,2010-12-29,Northeast Division,Corporate,Budget,Salaries,Expenditures,20200.0
2,3,2010-12-29,Northeast Division,Corporate,Budget,Payroll Taxes,Expenditures,2000.0
3,4,2010-12-29,Northeast Division,Corporate,Actual,Payroll Taxes,Expenditures,2208.0
4,5,2010-12-29,Northeast Division,Corporate,Actual,Employee Benefits,Expenditures,1546.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39409 entries, 0 to 39408
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   FinanceKey            39409 non-null  int64  
 1   FullDateAlternateKey  39409 non-null  object 
 2   OrganizationName      39409 non-null  object 
 3   DepartmentGroupName   39409 non-null  object 
 4   ScenarioName          39409 non-null  object 
 5   AccountDescription    39409 non-null  object 
 6   AccountType           39409 non-null  object 
 7   Amount                39409 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 2.4+ MB


In [5]:
data['FullDateAlternateKey']= pd.to_datetime(data['FullDateAlternateKey'])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39409 entries, 0 to 39408
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   FinanceKey            39409 non-null  int64         
 1   FullDateAlternateKey  39409 non-null  datetime64[ns]
 2   OrganizationName      39409 non-null  object        
 3   DepartmentGroupName   39409 non-null  object        
 4   ScenarioName          39409 non-null  object        
 5   AccountDescription    39409 non-null  object        
 6   AccountType           39409 non-null  object        
 7   Amount                39409 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 2.4+ MB


In [7]:
obj = pd.Series(data.Amount.head(),index=data.index.values[:5],name='Series')

In [8]:
obj

0    22080.0
1    20200.0
2     2000.0
3     2208.0
4     1546.0
Name: Series, dtype: float64

<a id="cell"></a>

In [9]:
print ('Index details')
print (obj.index)
print ('Values details')
print (obj.values)
print (obj.shape)
print (len(obj))
print (obj.dtype)
print (obj.name)

Index details
Int64Index([0, 1, 2, 3, 4], dtype='int64')
Values details
[22080. 20200.  2000.  2208.  1546.]
(5,)
5
float64
Series


<a id="cell1"></a>

In [10]:
# Selecting specific values based on its index
obj[4]

1546.0

In [11]:
# Select multiple values based on their index
obj[[1,3]]

1    20200.0
3     2208.0
Name: Series, dtype: float64

<a id="cell2"></a>

In [12]:
# Filtering data based on a given criteria
obj[obj>2000]

0    22080.0
1    20200.0
3     2208.0
Name: Series, dtype: float64

In [13]:
obj[(obj > 21000) | (obj < 1600)]

0    22080.0
4     1546.0
Name: Series, dtype: float64

<a id="cell3"></a>

In [14]:
obj.idxmax()

0

In [15]:
obj.idxmin()

4

In [16]:
obj.argmin()

4

<a id="cell4"></a>

In [17]:
# Checking boolean
2 in obj

True

In [18]:
2000 in obj.values

True

<a id="cell5"></a>

In [19]:
sql1 = """SELECT FF.FinanceKey,DO.OrganizationName,DA.AccountDescription, DA.AccountType,FF.Amount

FROM FactFinance as FF JOIN DimOrganization AS DO
ON FF.OrganizationKey = DO.OrganizationKey
JOIN DimAccount AS DA
ON FF.AccountKey = DA.AccountKey

"""

In [20]:
data1 = pd.read_sql(sql1,cnxn)
data1

,FinanceKey,OrganizationName,AccountDescription,AccountType,Amount
0,1,Northeast Division,Salaries,Expenditures,22080.0
1,2,Northeast Division,Salaries,Expenditures,20200.0
2,3,Northeast Division,Payroll Taxes,Expenditures,2000.0
3,4,Northeast Division,Payroll Taxes,Expenditures,2208.0
4,5,Northeast Division,Employee Benefits,Expenditures,1546.0
...,...,...,...,...,...
39404,39405,Australia,Partner Capital,Liabilities,225000.0
39405,39406,Australia,Additional Paid In Capital,Liabilities,-225000.0
39406,39407,Australia,Amortization of Goodwill,Expenditures,-1957.7
39407,39408,Australia,Other Income,Revenue,17619.3


In [21]:
data1.drop_duplicates(['OrganizationName'], inplace = True)

In [22]:
data1

,FinanceKey,OrganizationName,AccountDescription,AccountType,Amount
0,1,Northeast Division,Salaries,Expenditures,22080.00
188,189,Northwest Division,Salaries,Expenditures,43120.00
329,330,Central Division,Salaries,Expenditures,33920.00
470,471,Southeast Division,Salaries,Expenditures,20800.00
801,802,Southwest Division,Salaries,Expenditures,37080.00
942,943,Canadian Division,Salaries,Expenditures,48000.46
14217,14218,France,Salaries,Expenditures,20229.86
14390,14391,Germany,Salaries,Expenditures,1909.31
26701,26702,Australia,Salaries,Expenditures,19305.12


In [23]:
sdata = {'Northeast Division': 22080, 'Northwest Division': 43120, 'Central Division': 33920, 'Southeast Division': 20800}
obj3 = pd.Series(sdata)
print (obj3)

Northeast Division    22080
Northwest Division    43120
Central Division      33920
Southeast Division    20800
dtype: int64


In [24]:
regions = ['Northeast Division', 'Northwest Division', 'Central Division', 'Canadian Division'] 

In [25]:
obj4 = pd.Series(sdata, index = regions) 

#it takes the values of sdata but using as index the states data
# if there is a match it populates the value field

In [26]:
obj4

Northeast Division    22080.0
Northwest Division    43120.0
Central Division      33920.0
Canadian Division         NaN
dtype: float64

<a id="cell7"></a>

In [27]:
# Pandas functions
print (pd.isnull(obj4))
print (pd.notnull(obj4))

Northeast Division    False
Northwest Division    False
Central Division      False
Canadian Division      True
dtype: bool
Northeast Division     True
Northwest Division     True
Central Division       True
Canadian Division     False
dtype: bool


<a id="cell6"></a>

In [28]:
# Data manipulation
obj3 * 2

Northeast Division    44160
Northwest Division    86240
Central Division      67840
Southeast Division    41600
dtype: int64

In [29]:
np.exp(obj3)

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


Northeast Division    inf
Northwest Division    inf
Central Division      inf
Southeast Division    inf
dtype: float64

In [30]:
# Combining Series
obj3 + obj4

Canadian Division         NaN
Central Division      67840.0
Northeast Division    44160.0
Northwest Division    86240.0
Southeast Division        NaN
dtype: float64

<a id="cell8"></a>

In [31]:
obj4.name = 'Geographies'

In [32]:
obj4.index.name = 'Regions'

In [33]:
obj4

Regions
Northeast Division    22080.0
Northwest Division    43120.0
Central Division      33920.0
Canadian Division         NaN
Name: Geographies, dtype: float64

### Changing numbers format in Pandas Dataframe
<a id="cell9"></a>

The following example already shows some major features of the `DataFrame` class when it comes to storing data:

* **data**: data itself can be provided in different shapes and types
* **labels**: data is organized in columns which can have custom names
* **index**: there is an index that can take on different formats

In [34]:
df = pd.DataFrame(np.random.randn(2, 2))
df

,0,1
0,-0.054047,-1.476974
1,0.372948,0.741765


In [35]:
df[1] = df[1].map('${:,.2f}'.format)
df[1]

0    $-1.48
1     $0.74
Name: 1, dtype: object

In [36]:
df[0] = pd.Series(["{0:.2f}%".format(val * 100) for val in df[0]], 
                  index = df.index)
df[0]

0    -5.40%
1    37.29%
Name: 0, dtype: object

### Advance Formatting

In [37]:
total = 1022032.79
avgrate = 4.060474510561586
life = 6.9547965672323002

In [38]:
sbk = pd.DataFrame({'Values':['${:,.0f}'.format(total),
                              '{:,.2f}%'.format(avgrate),
                              '{:,.2f}'.format(life)]},
                   index = ['Total',
                            'Avg. Rate',
                            'Avg. Life (yrs.)'])

In [39]:
sbk

,Values
Total,"$1,022,033"
Avg. Rate,4.06%
Avg. Life (yrs.),6.95


<a id="cell10"></a>

In [40]:
df1 = {'Region': ['Northeast Division', 'Northwest Division','Central Division', 'Southeast Division', 'Southeast Division'],
       'FinanceKey': [1,189,330,471,802],
       'Amount' : [22080., 43120., 33920., 20800., 37080.]}

In [41]:
frame = pd.DataFrame(df1)
frame

,Region,FinanceKey,Amount
0,Northeast Division,1,22080.0
1,Northwest Division,189,43120.0
2,Central Division,330,33920.0
3,Southeast Division,471,20800.0
4,Southeast Division,802,37080.0


In [42]:
frame2 = pd.DataFrame(df1, index = ['one','two', 'three', 'four', 'five'], 
                   columns = ['Region', 'FinanceKey', 'Amount','Code']) 
# Warning = the names of the columns should match the names on df1
# otherwise it wont map the values
frame2

,Region,FinanceKey,Amount,Code
one,Northeast Division,1,22080.0,NaN
two,Northwest Division,189,43120.0,NaN
three,Central Division,330,33920.0,NaN
four,Southeast Division,471,20800.0,NaN
five,Southeast Division,802,37080.0,NaN


<a id="cell11"></a>

In [43]:
frame2.columns # it retrieves the name of the columns

Index(['Region', 'FinanceKey', 'Amount', 'Code'], dtype='object')

In [44]:
frame2.index # it retrieves the name of the rows

Index(['one', 'two', 'three', 'four', 'five'], dtype='object')

In [45]:
frame2['Region'] # it retrieves the values of the requested column

one      Northeast Division
two      Northwest Division
three      Central Division
four     Southeast Division
five     Southeast Division
Name: Region, dtype: object

<a id="cell12"></a>

In [46]:
frame2.loc['two']

Region        Northwest Division
FinanceKey                   189
Amount                   43120.0
Code                         NaN
Name: two, dtype: object

<a id="cell13"></a>

In [47]:
frame2['Code'] = 16
frame2

,Region,FinanceKey,Amount,Code
one,Northeast Division,1,22080.0,16
two,Northwest Division,189,43120.0,16
three,Central Division,330,33920.0,16
four,Southeast Division,471,20800.0,16
five,Southeast Division,802,37080.0,16


#### Important: DataFrame is case sensitive!! small mistakes have huge impact

In [48]:
frame2['code'] = 16
frame2

,Region,FinanceKey,Amount,Code,code
one,Northeast Division,1,22080.0,16,16
two,Northwest Division,189,43120.0,16,16
three,Central Division,330,33920.0,16,16
four,Southeast Division,471,20800.0,16,16
five,Southeast Division,802,37080.0,16,16


<a id="cell14"></a>

In [49]:
del frame2['code']
frame2

,Region,FinanceKey,Amount,Code
one,Northeast Division,1,22080.0,16
two,Northwest Division,189,43120.0,16
three,Central Division,330,33920.0,16
four,Southeast Division,471,20800.0,16
five,Southeast Division,802,37080.0,16


In [50]:
# filling a column with a sequence of numbers
frame2['Sequence'] = np.arange(5)
frame2

,Region,FinanceKey,Amount,Code,Sequence
one,Northeast Division,1,22080.0,16,0
two,Northwest Division,189,43120.0,16,1
three,Central Division,330,33920.0,16,2
four,Southeast Division,471,20800.0,16,3
five,Southeast Division,802,37080.0,16,4


<a id="cell15"></a>

In [51]:
# Column where the boolean value True or False will depend on a condition
frame2['KeyArea'] = frame2.Region == 'Central Division'
frame2

,Region,FinanceKey,Amount,Code,Sequence,KeyArea
one,Northeast Division,1,22080.0,16,0,False
two,Northwest Division,189,43120.0,16,1,False
three,Central Division,330,33920.0,16,2,True
four,Southeast Division,471,20800.0,16,3,False
five,Southeast Division,802,37080.0,16,4,False


### Change column position in frame
<a id="cell101"></a>

In [52]:
frame2.columns.values

array(['Region', 'FinanceKey', 'Amount', 'Code', 'Sequence', 'KeyArea'],
      dtype=object)

In [53]:
frame2 = frame2[['KeyArea', 'Code', 'Sequence', 'Region', 'FinanceKey', 'Amount']]

In [54]:
frame2

,KeyArea,Code,Sequence,Region,FinanceKey,Amount
one,False,16,0,Northeast Division,1,22080.0
two,False,16,1,Northwest Division,189,43120.0
three,True,16,2,Central Division,330,33920.0
four,False,16,3,Southeast Division,471,20800.0
five,False,16,4,Southeast Division,802,37080.0


### Sorting values in Dataframe
<a id="cell102"></a>

In [55]:
frame2.sort_values(by ='Region', ascending= False)

,KeyArea,Code,Sequence,Region,FinanceKey,Amount
four,False,16,3,Southeast Division,471,20800.0
five,False,16,4,Southeast Division,802,37080.0
two,False,16,1,Northwest Division,189,43120.0
one,False,16,0,Northeast Division,1,22080.0
three,True,16,2,Central Division,330,33920.0


#### For array sorting numpy.sort could be of great use
<a id="cell103"></a>

In [56]:
np.sort(frame2.Sequence.values)

array([0, 1, 2, 3, 4])

### Assigning values to specific cells in the dataframe

In [57]:
codes = pd.Series([12, 18, 20], index =['two', 'four', 'five'])

In [58]:
frame2['Code'] = codes # it will assign values matching using
frame2               # the index of the val series

,KeyArea,Code,Sequence,Region,FinanceKey,Amount
one,False,NaN,0,Northeast Division,1,22080.0
two,False,12.0,1,Northwest Division,189,43120.0
three,True,NaN,2,Central Division,330,33920.0
four,False,18.0,3,Southeast Division,471,20800.0
five,False,20.0,4,Southeast Division,802,37080.0


### It is important to see that it replaces not only the values in the series but it also deletes the rest of the values

In [59]:
Sales = {'Northeast Division': {2018 : 4.6, 2019: 5.6}, 
       'Central Division': {2018: 5.2, 2019: 6.7, 2020:3.6}}
# layers of Dictionaries

In [60]:
frame3 = pd.DataFrame(Sales)
frame3

,Northeast Division,Central Division
2018,4.6,5.2
2019,5.6,6.7
2020,NaN,3.6


In [61]:
frame3.T

,2018,2019,2020
Northeast Division,4.6,5.6,NaN
Central Division,5.2,6.7,3.6


In [62]:
pd.DataFrame(Sales,index = [2017,2018,2019])

,Northeast Division,Central Division
2017,NaN,NaN
2018,4.6,5.2
2019,5.6,6.7


### Selectiong data left to right & right to left

In [63]:
pdata = {'Central Division' : frame3['Central Division'][:-1], 'Central Division': frame3['Central Division'][:2]}

In [64]:
frame3['Central Division'][:-1]

2018    5.2
2019    6.7
Name: Central Division, dtype: float64

In [65]:
pd.DataFrame(pdata)

,Central Division
2018,5.2
2019,6.7


<a id="cell16"></a>

In [66]:
frame3.index.name = 'years'; frame3.columns.name = 'Region' 
# good way to pass two statements on the same line
frame3

Region,Northeast Division,Central Division
years,,
2018,4.6,5.2
2019,5.6,6.7
2020,NaN,3.6


<a id="cell17"></a>

In [67]:
frame3.values # it gets you the values only

array([[4.6, 5.2],
       [5.6, 6.7],
       [nan, 3.6]])

In [68]:
obj5 = pd.Series(np.arange(4) , index = np.arange(4))
obj5

0    0
1    1
2    2
3    3
dtype: int32

<a id="cell18"></a>

In [69]:
obj6 = obj5.reindex([2,4,3,1])
obj6

2    2.0
4    NaN
3    3.0
1    1.0
dtype: float64

In [70]:
obj5.reindex([2,4,3,1],fill_value = 0)

2    2
4    0
3    3
1    1
dtype: int32

In [71]:
obj7 = pd.Series(['Northeast', 'Southeast','Central'], index = [1,2,3])
obj7

1    Northeast
2    Southeast
3      Central
dtype: object

In [72]:
obj7.reindex(range(6), method = 'ffill') 
#it defines the length of the range followed by the method selected
#for backward filling use --> bfill

0          NaN
1    Northeast
2    Southeast
3      Central
4      Central
5      Central
dtype: object

In [73]:
datas = pd.DataFrame(np.arange(10,210,10).reshape((5,4)),
                     index = ['Northeast','Northwest','Central','Southeast', 'Southwest'],
                     columns = [16,17,18,19])


In [74]:
datas

,16,17,18,19
Northeast,10,20,30,40
Northwest,50,60,70,80
Central,90,100,110,120
Southeast,130,140,150,160
Southwest,170,180,190,200


<a id="cell19"></a>

In [75]:
datas.drop(['Southeast', 'Southwest'])

,16,17,18,19
Northeast,10,20,30,40
Northwest,50,60,70,80
Central,90,100,110,120


In [76]:
datas.drop([17,18], axis =1)

,16,19
Northeast,10,40
Northwest,50,80
Central,90,120
Southeast,130,160
Southwest,170,200
